# teste de modelo com arvóre de decisão

In [13]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


## passo a passo

 - Verificar existência de dados nulos
 - Avaliar se existem colunas que podem ser removidas
 - Identificar as variáveis categoricas e as alvo.
 - Codificar as colunas cartegoricas passiveis de codificação

In [14]:
df = pd.read_csv(r'/home/lipe/projetos/case-Fm2c/data/TurnOver_Analise.csv', sep=';')

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19671 entries, 0 to 19670
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Cargo             19671 non-null  object
 1   GRUPO EMPRESA     19671 non-null  object
 2   Matricula         19671 non-null  object
 3   Idade             19671 non-null  int64 
 4   Estado            19671 non-null  object
 5   Classif. Salário  19671 non-null  object
 6   Situação          19671 non-null  object
 7   Tempo de Empresa  19671 non-null  object
 8   Vale Aliment.     19671 non-null  object
 9   Vale Refei.       19671 non-null  object
 10  Vale Transp.      19671 non-null  object
 11  Qtd. Dependentes  19671 non-null  object
 12  Conv. Médico      19671 non-null  object
 13  Conv. Odonto      19671 non-null  object
 14  Qtd. Promoções    19671 non-null  object
dtypes: int64(1), object(14)
memory usage: 2.3+ MB


In [16]:
# Removendo coluna Matricula
df.drop(columns='Matricula', inplace=True)

In [17]:
'''
Coluna Situação = variável alvo
Realizar remapeamento dos valores da coluna, Ativos = 1 e Desligados = 2
'''

df.Situação = df.Situação.map({'Ativos': 1, 'Desligados': 0})

In [19]:
'''
Colunas Vale(Alimentação, refeição, transporte)
Colunas Qtd(Dependentes, promoções)
Colunas Conv(Médico, Odonto)

Serão tranformados em inteiros. Apenas a coluna Qtd de dependentes vai representar a quantidade real de dependentes do colaborador
As outras os valores 1 correspondem a Tem aquele benefício e 0 como Não tem o benefício
'''

df['Tempo de Empresa'] = df['Tempo de Empresa'].str.replace(',','.')
df['Vale Aliment.'] = df['Vale Aliment.'].str.replace(',','.')
df['Vale Refei.'] = df['Vale Refei.'].str.replace(',','.')
df['Vale Transp.'] = df['Vale Transp.'].str.replace(',','.')
df['Qtd. Dependentes'] = df['Qtd. Dependentes'].str.replace(',','.')
df['Conv. Médico'] = df['Conv. Médico'].str.replace(',','.')
df['Conv. Odonto'] = df['Conv. Odonto'].str.replace(',','.')
df['Qtd. Promoções'] = df['Qtd. Promoções'].str.replace(',','.')

df['Tempo de Empresa'] = df['Tempo de Empresa'].astype(float)
df['Vale Aliment.'] = df['Vale Aliment.'].astype(float)
df['Vale Refei.'] = df['Vale Refei.'].astype(float)
df['Vale Transp.'] = df['Vale Transp.'].astype(float)
df['Qtd. Dependentes'] = df['Qtd. Dependentes'].astype(float)
df['Conv. Médico'] = df['Conv. Médico'].astype(float)
df['Conv. Odonto'] = df['Conv. Odonto'].astype(float)
df['Qtd. Promoções'] = df['Qtd. Promoções'].astype(float)

df['Tempo de Empresa'] = df['Tempo de Empresa'].astype(int)
df['Vale Aliment.'] = df['Vale Aliment.'].astype(int)
df['Vale Refei.'] = df['Vale Refei.'].astype(int)
df['Vale Transp.'] = df['Vale Transp.'].astype(int)
df['Qtd. Dependentes'] = df['Qtd. Dependentes'].astype(int)
df['Conv. Médico'] = df['Conv. Médico'].astype(int)
df['Conv. Odonto'] = df['Conv. Odonto'].astype(int)
df['Qtd. Promoções'] = df['Qtd. Promoções'].astype(int)

In [20]:
''' 
tratamento de colunas que possume dados em formato string, limitados até três tipos de dados diferentes
Colunas Grupo empresa e classif Salário
'''

df['GRUPO EMPRESA'] = df['GRUPO EMPRESA'].map({'EMPRESA A': 0, 'EMPRESA B': 1, 'EMPRESA C': 2})
df['Classif. Salário'] = df['Classif. Salário'].map({'Baixo': 0, 'Médio': 1, 'Alto': 2})


In [21]:

'''
Transformar o restante das colunas que tem mais de 3 tipos de dados (Cargo, Estado)
Verificar a existência de dados nulos anso de aplicar o encoder
'''
encoder = LabelEncoder()

#df['Cargo'] = df['Cargo'].str.strip()

df['Cargo'] = encoder.fit_transform(df['Cargo'])
df['Estado'] = encoder.fit_transform(df['Estado'])


In [37]:
'''
Separar variável alvo das categóricas
Separa conjunto de dados para treinamento e teste
Definir semente para controlar a variação randomica dos dados entre a separação de treino e teste
'''
X = df[['Cargo', 'GRUPO EMPRESA', 'Idade', 'Estado', 'Classif. Salário', 'Tempo de Empresa', 'Vale Aliment.', 'Vale Refei.', 'Vale Transp.', 'Qtd. Dependentes', 'Conv. Médico', 'Conv. Odonto', 'Qtd. Promoções']]
y = df.Situação

np.random.seed(5)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [ ]:
cls  = DecisionTreeClassifier()
cls.fit(X=X_train, y=y_train)

DecisionTreeClassifier()

In [39]:
print(f'Acurácia: {cls.score(X_test, y_test): .2%}')

Acurácia:  77.19%
